<a href="https://colab.research.google.com/github/Pegasus-nathanael/Test_Nathan/blob/main/Scrapping_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### 1 Work on scrapping. nj